# 📊 Evaluación de Modelos de Producción - DeAcero Steel Price Predictor

Este notebook evalúa los **4 mejores modelos** identificados en el A/B Testing para predicción del precio de cierre t+1 de la varilla corrugada LME.

## 🎯 Modelos a Evaluar:
1. **MIDAS V2 Híbrida** - MAPE: 0.49%, R²: 0.975 (Mejor overall)
2. **MIDAS V2 Régimen** - MAPE: 1.38%, R²: 0.789
3. **XGBoost V2 Régimen** - MAPE: 1.48%, R²: 0.787  
4. **XGBoost V2 Híbrida** - MAPE: 1.52%, R²: 0.774

## 📋 Objetivos de la Evaluación:
1. **Cargar modelos entrenados** desde `models/test/`
2. **Preparar datos de prueba** con las mismas transformaciones
3. **Generar predicciones** para diferentes horizontes temporales
4. **Evaluar métricas** (MAPE, MAE, RMSE, R², Hit Rate, Directional Accuracy)
5. **Análisis de residuos** y diagnósticos estadísticos
6. **Comparación visual** de predicciones vs valores reales
7. **Selección del modelo final** para producción
8. **Generar reporte ejecutivo** de evaluación


In [19]:
# 🔧 CONFIGURACIÓN INICIAL Y LIBRERÍAS

import sys
import os
sys.path.append('../')

# Data manipulation
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Model loading
import pickle
import json
import joblib

# ML & Statistics
from sklearn.metrics import (
    mean_absolute_error, 
    mean_squared_error, 
    r2_score,
    mean_absolute_percentage_error,
    explained_variance_score
)
from sklearn.preprocessing import RobustScaler
from scipy import stats
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')

# Configuración de visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

print("=" * 80)
print("📊 SISTEMA DE EVALUACIÓN DE MODELOS INICIALIZADO")
print("=" * 80)
print(f"📅 Fecha de evaluación: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"📁 Directorio de trabajo: {os.getcwd()}")
print(f"🐍 Python version: {sys.version.split()[0]}")


📊 SISTEMA DE EVALUACIÓN DE MODELOS INICIALIZADO
📅 Fecha de evaluación: 2025-09-29 22:10:24
📁 Directorio de trabajo: /Users/franciscojavierriverapaleo/test_gerencia/deacero_steel_price_predictor/notebooks
🐍 Python version: 3.13.7


In [20]:
# 📂 CARGAR LOS 4 MEJORES MODELOS DESDE models/test/

print("=" * 80)
print("📂 CARGANDO MODELOS DE PRODUCCIÓN")
print("=" * 80)

# Directorio de modelos
models_dir = '../models/test'

# Definir los 4 modelos a cargar
models_to_load = {
    'MIDAS_V2_hibrida': {
        'file': 'MIDAS_V2_hibrida.pkl',
        'description': 'MIDAS V2 Híbrida - Mejor modelo overall',
        'expected_mape': 0.49,
        'expected_r2': 0.975
    },
    'MIDAS_V2_regime': {
        'file': 'MIDAS_V2_regime.pkl',
        'description': 'MIDAS V2 Régimen - Robusto a cambios de mercado',
        'expected_mape': 1.38,
        'expected_r2': 0.789
    },
    'XGBoost_V2_regime': {
        'file': 'XGBoost_V2_regime.pkl',
        'description': 'XGBoost V2 Régimen - Captura no-linealidades',
        'expected_mape': 1.48,
        'expected_r2': 0.787
    },
    'XGBoost_V2_hibrida': {
        'file': 'XGBoost_V2_hibrida.pkl',
        'description': 'XGBoost V2 Híbrida - Balance autorregresivo',
        'expected_mape': 1.52,
        'expected_r2': 0.774
    }
}

# Diccionario para almacenar los modelos cargados
loaded_models = {}
model_configs = {}

print(f"\n📁 Directorio de modelos: {models_dir}")
print("-" * 60)

# Cargar cada modelo
for model_name, model_info in models_to_load.items():
    model_path = os.path.join(models_dir, model_info['file'])
    
    print(f"\n🔄 Cargando: {model_name}")
    print(f"   📄 Archivo: {model_info['file']}")
    
    try:
        # Cargar modelo pickle
        with open(model_path, 'rb') as f:
            model_data = pickle.load(f)
        
        loaded_models[model_name] = model_data
        
        # Verificar estructura del modelo
        print(f"   ✅ Modelo cargado exitosamente")
        print(f"   📝 {model_info['description']}")
        print(f"   📊 MAPE esperado: {model_info['expected_mape']:.2f}%")
        print(f"   📊 R² esperado: {model_info['expected_r2']:.3f}")
        
        # Verificar componentes del modelo
        if isinstance(model_data, dict):
            components = list(model_data.keys())
            print(f"   🔧 Componentes: {', '.join(components[:5])}")
            
            # Extraer métricas si existen
            if 'test_metrics' in model_data:
                actual_metrics = model_data['test_metrics']
                print(f"   📈 Métricas reales:")
                print(f"      MAPE: {actual_metrics.get('mape', 'N/A'):.2f}%")
                print(f"      R²: {actual_metrics.get('r2', 'N/A'):.3f}")
                print(f"      RMSE: {actual_metrics.get('rmse', 'N/A'):.4f}")
        
        # Cargar configuración JSON si existe
        config_path = os.path.join(models_dir, f"{model_name.replace('_V2', '_v2')}_config.json")
        if os.path.exists(config_path):
            with open(config_path, 'r') as f:
                model_configs[model_name] = json.load(f)
            print(f"   📋 Configuración cargada")
        
    except FileNotFoundError:
        print(f"   ❌ Error: Archivo no encontrado - {model_path}")
        print(f"   💡 Verifica que el modelo haya sido entrenado y guardado")
        
    except Exception as e:
        print(f"   ❌ Error al cargar: {str(e)[:100]}")

# Resumen de carga
print("\n" + "=" * 80)
print("📊 RESUMEN DE CARGA")
print("=" * 80)
print(f"✅ Modelos cargados exitosamente: {len(loaded_models)}/{len(models_to_load)}")

if len(loaded_models) > 0:
    print("\n🏆 Modelos disponibles para evaluación:")
    for i, model_name in enumerate(loaded_models.keys(), 1):
        print(f"   {i}. {model_name}")
else:
    print("\n⚠️ No se pudieron cargar modelos. Verifica que:")
    print("   1. Los modelos hayan sido entrenados (ejecutar notebook 03_AB_TESTING)")
    print("   2. Los archivos .pkl estén en models/test/")
    print("   3. Los nombres de archivo coincidan")


📂 CARGANDO MODELOS DE PRODUCCIÓN

📁 Directorio de modelos: ../models/test
------------------------------------------------------------

🔄 Cargando: MIDAS_V2_hibrida
   📄 Archivo: MIDAS_V2_hibrida.pkl
   ✅ Modelo cargado exitosamente
   📝 MIDAS V2 Híbrida - Mejor modelo overall
   📊 MAPE esperado: 0.49%
   📊 R² esperado: 0.975
   🔧 Componentes: model, scalers, midas_features, best_params, train_metrics
   📈 Métricas reales:
      MAPE: 0.26%
      R²: 0.974
      RMSE: 2.4473
   📋 Configuración cargada

🔄 Cargando: MIDAS_V2_regime
   📄 Archivo: MIDAS_V2_regime.pkl
   ✅ Modelo cargado exitosamente
   📝 MIDAS V2 Régimen - Robusto a cambios de mercado
   📊 MAPE esperado: 1.38%
   📊 R² esperado: 0.789
   🔧 Componentes: model, scalers, midas_features, best_params, train_metrics
   📈 Métricas reales:
      MAPE: 2.63%
      R²: -0.370
      RMSE: 17.7936
   📋 Configuración cargada

🔄 Cargando: XGBoost_V2_regime
   📄 Archivo: XGBoost_V2_regime.pkl
   ✅ Modelo cargado exitosamente
   📝 XGBoost 

In [21]:
# 📊 CARGAR DATOS PARA EVALUACIÓN

print("=" * 80)
print("📊 CARGANDO DATOS PARA EVALUACIÓN")
print("=" * 80)

# Cargar datos consolidados
data_path_daily = '../data/processed/daily_time_series/daily_series_consolidated_latest.csv'
data_path_monthly = '../data/processed/monthly_time_series/monthly_series_consolidated_latest.csv'

print("\n📂 Cargando datos diarios...")
try:
    daily_data = pd.read_csv(data_path_daily, index_col='fecha', parse_dates=True)
    for col in daily_data.columns:
        daily_data[col] = daily_data[col].interpolate(method='linear', limit_direction='both')
    print(f"   ✅ Datos diarios cargados: {daily_data.shape}")
    print(f"   📅 Período: {daily_data.index[0].strftime('%Y-%m-%d')} a {daily_data.index[-1].strftime('%Y-%m-%d')}")
    print(f"   📊 Variables disponibles: {len(daily_data.columns)}")
except Exception as e:
    print(f"   ❌ Error al cargar datos diarios: {str(e)[:100]}")
    daily_data = pd.DataFrame()

print("\n📂 Cargando datos mensuales...")
try:
    monthly_data = pd.read_csv(data_path_monthly, index_col='fecha', parse_dates=True)
    for col in monthly_data.columns:
        monthly_data[col] = monthly_data[col].interpolate(method='linear', limit_direction='both')
    print(f"   ✅ Datos mensuales cargados: {monthly_data.shape}")
    print(f"   📅 Período: {monthly_data.index[0].strftime('%Y-%m')} a {monthly_data.index[-1].strftime('%Y-%m')}")
except Exception as e:
    print(f"   ⚠️ Datos mensuales no disponibles: {str(e)[:100]}")
    monthly_data = pd.DataFrame()

# Definir variable objetivo
TARGET_VAR = 'precio_varilla_lme'

if TARGET_VAR in daily_data.columns:
    print(f"\n🎯 Variable objetivo: {TARGET_VAR}")
    print(f"   Último precio: ${daily_data[TARGET_VAR].iloc[-1]:.2f}")
    print(f"   Precio promedio: ${daily_data[TARGET_VAR].mean():.2f}")
    print(f"   Desviación estándar: ${daily_data[TARGET_VAR].std():.2f}")
    print(f"   Min/Max: ${daily_data[TARGET_VAR].min():.2f} / ${daily_data[TARGET_VAR].max():.2f}")
else:
    print(f"\n⚠️ Variable objetivo '{TARGET_VAR}' no encontrada en los datos")

# Definir las combinaciones de features
feature_combinations = {
    'hibrida': ['precio_varilla_lme_lag_1', 'volatility_20', 'iron', 
                'coking', 'commodities', 'VIX'],
    'regime': ['iron', 'coking', 'steel', 'VIX', 'sp500', 
               'tasa_interes_banxico'],
    'fundamental': ['iron', 'coking', 'gas_natural', 'aluminio_lme', 
                    'commodities']
}

print("\n📋 Combinaciones de variables definidas:")
for combo_name, features in feature_combinations.items():
    available = sum([1 for f in features if f in daily_data.columns or f == 'precio_varilla_lme_lag_1' or f == 'volatility_20'])
    print(f"   {combo_name}: {available}/{len(features)} variables disponibles")


📊 CARGANDO DATOS PARA EVALUACIÓN

📂 Cargando datos diarios...
   ✅ Datos diarios cargados: (1498, 28)
   📅 Período: 2020-01-02 a 2025-09-29
   📊 Variables disponibles: 28

📂 Cargando datos mensuales...
   ✅ Datos mensuales cargados: (68, 9)
   📅 Período: 2020-01 a 2025-08

🎯 Variable objetivo: precio_varilla_lme
   Último precio: $540.50
   Precio promedio: $494.60
   Desviación estándar: $29.40
   Min/Max: $422.12 / $580.70

📋 Combinaciones de variables definidas:
   hibrida: 6/6 variables disponibles
   regime: 6/6 variables disponibles
   fundamental: 5/5 variables disponibles


In [22]:
# 🛠️ FUNCIONES DE PREPARACIÓN Y EVALUACIÓN

print("=" * 80)
print("🛠️ DEFINIENDO FUNCIONES DE EVALUACIÓN")
print("=" * 80)

def create_features(df, target_var='precio_varilla_lme'):
    """Crear features de ingeniería para los modelos"""
    features_df = df.copy()
    
    # 1. Lags de la variable objetivo
    for lag in [1, 2, 3, 5, 10, 20]:
        features_df[f'{target_var}_lag_{lag}'] = df[target_var].shift(lag)
    
    # 2. Medias móviles
    for window in [5, 10, 20, 50]:
        features_df[f'{target_var}_ma_{window}'] = df[target_var].rolling(window=window).mean()
    
    # 3. Volatilidad
    for window in [5, 10, 20]:
        features_df[f'volatility_{window}'] = df[target_var].pct_change().rolling(window=window).std()
    
    # 4. Returns
    features_df['return_1d'] = df[target_var].pct_change()
    features_df['return_5d'] = df[target_var].pct_change(5)
    
    return features_df

def prepare_data_for_prediction(df, feature_list, target_var='precio_varilla_lme'):
    """Preparar datos para predicción con las features específicas"""
    # Crear features
    df_features = create_features(df, target_var)
    
    # Seleccionar features disponibles
    available_features = []
    for feature in feature_list:
        if feature in df_features.columns:
            available_features.append(feature)
        elif feature in df.columns:
            df_features[feature] = df[feature]
            available_features.append(feature)
    
    # Preparar X e y
    X = df_features[available_features].copy()
    y = df[target_var].copy()
    
    # Eliminar NaN
    valid_idx = ~(X.isnull().any(axis=1) | y.isnull())
    X = X[valid_idx]
    y = y[valid_idx]
    
    # Agregar precio actual como feature crítica
    X['current_price'] = y.values
    X['price_ma20'] = y.rolling(20).mean().values
    X['price_std20'] = y.rolling(20).std().values
    
    return X, y

def calculate_metrics(y_true, y_pred, model_name="Model"):
    """Calcular métricas de evaluación"""
    # Asegurar que son arrays numpy
    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()
    
    # Filtrar NaN si existen
    mask = ~(np.isnan(y_true) | np.isnan(y_pred))
    y_true = y_true[mask]
    y_pred = y_pred[mask]
    
    metrics = {
        'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
        'mae': mean_absolute_error(y_true, y_pred),
        'mape': mean_absolute_percentage_error(y_true, y_pred) * 100,
        'r2': r2_score(y_true, y_pred),
        'explained_variance': explained_variance_score(y_true, y_pred)
    }
    
    # Directional accuracy
    if len(y_true) > 1:
        y_true_diff = np.diff(y_true)
        y_pred_diff = np.diff(y_pred)
        directional_accuracy = np.mean(np.sign(y_true_diff) == np.sign(y_pred_diff)) * 100
        metrics['directional_accuracy'] = directional_accuracy
    
    # Hit rate ±2%
    threshold = 0.02
    within_threshold = np.abs((y_pred - y_true) / y_true) <= threshold
    metrics['hit_rate_2pct'] = np.mean(within_threshold) * 100
    
    return metrics

def make_predictions(model_data, X_test, model_type='XGBoost'):
    """Hacer predicciones según el tipo de modelo"""
    try:
        # Primero verificar si hay predicciones pre-guardadas
        if 'predictions' in model_data:
            if 'test' in model_data['predictions']:
                print(f"   📌 Usando predicciones pre-guardadas")
                return model_data['predictions']['test']
        
        # Si no hay predicciones guardadas, intentar predecir con el modelo
        if 'model' in model_data and model_data['model'] is not None:
            model = model_data['model']
            
            # Para MIDAS, usar las features MIDAS específicas si existen
            if model_type == 'MIDAS' and 'midas_features' in model_data:
                if 'test' in model_data['midas_features']:
                    print(f"   📌 Usando features MIDAS pre-procesadas")
                    X_test = model_data['midas_features']['test']
                    # Asegurar que X_test sea DataFrame
                    if not isinstance(X_test, pd.DataFrame):
                        X_test = pd.DataFrame(X_test)
            
            # Verificar y ajustar nombres de columnas si es necesario
            if hasattr(model, 'feature_names_in_'):
                expected_features = model.feature_names_in_
                print(f"   📌 Features esperadas: {len(expected_features)}")
                
                # Si X_test es un DataFrame, verificar columnas
                if isinstance(X_test, pd.DataFrame):
                    current_features = X_test.columns.tolist()
                    
                    # Si las columnas no coinciden, intentar reordenar o crear dummy features
                    if set(current_features) != set(expected_features):
                        print(f"   ⚠️ Discrepancia en features. Ajustando...")
                        
                        # Crear DataFrame con las columnas esperadas
                        X_test_adjusted = pd.DataFrame(index=X_test.index)
                        
                        for feat in expected_features:
                            if feat in X_test.columns:
                                X_test_adjusted[feat] = X_test[feat]
                            else:
                                # Si la feature no existe, usar ceros o la media
                                print(f"      - Feature faltante: {feat}, usando 0")
                                X_test_adjusted[feat] = 0
                        
                        X_test = X_test_adjusted[expected_features]  # Asegurar orden correcto
                else:
                    # Si X_test no es DataFrame, convertirlo
                    if len(X_test.shape) == 2 and X_test.shape[1] == len(expected_features):
                        X_test = pd.DataFrame(X_test, columns=expected_features)
                    else:
                        print(f"   ⚠️ Dimensiones incorrectas: {X_test.shape} vs esperado: {len(expected_features)}")
            
            # Escalar datos si hay scalers
            if 'scalers' in model_data:
                scaler_X = model_data['scalers'].get('X')
                scaler_y = model_data['scalers'].get('y')
                
                if scaler_X is not None:
                    # Convertir a numpy array si es necesario
                    if isinstance(X_test, pd.DataFrame):
                        X_test_values = X_test.values
                    else:
                        X_test_values = X_test
                    
                    X_test_scaled = scaler_X.transform(X_test_values)
                else:
                    X_test_scaled = X_test
                
                # Hacer predicción
                y_pred_scaled = model.predict(X_test_scaled)
                
                # Desescalar si es necesario
                if scaler_y is not None:
                    y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()
                else:
                    y_pred = y_pred_scaled
            else:
                # Sin escalamiento
                y_pred = model.predict(X_test)
            
            return y_pred
        
        return None
        
    except Exception as e:
        print(f"   ⚠️ Error en predicción: {str(e)}")
        import traceback
        print(f"   📍 Traceback: {traceback.format_exc()[:500]}")
        return None

print("✅ Funciones de evaluación definidas:")
print("   • create_features: Genera features de ingeniería")
print("   • prepare_data_for_prediction: Prepara datos para predicción")
print("   • calculate_metrics: Calcula RMSE, MAE, MAPE, R², etc.")
print("   • make_predictions: Genera predicciones según el tipo de modelo")


🛠️ DEFINIENDO FUNCIONES DE EVALUACIÓN
✅ Funciones de evaluación definidas:
   • create_features: Genera features de ingeniería
   • prepare_data_for_prediction: Prepara datos para predicción
   • calculate_metrics: Calcula RMSE, MAE, MAPE, R², etc.
   • make_predictions: Genera predicciones según el tipo de modelo


In [23]:
# 🔍 DIAGNÓSTICO: VERIFICAR ESTRUCTURA DE MODELOS CARGADOS

print("=" * 80)
print("🔍 DIAGNÓSTICO DE MODELOS CARGADOS")
print("=" * 80)

for model_name, model_data in loaded_models.items():
    print(f"\n📦 {model_name}:")
    print(f"   Tipo de datos: {type(model_data)}")
    
    if isinstance(model_data, dict):
        print(f"   Claves disponibles: {list(model_data.keys())}")
        
        # Verificar si tiene modelo
        if 'model' in model_data:
            model = model_data['model']
            print(f"   Tipo de modelo: {type(model).__name__}")
            
            # Verificar features del modelo
            if hasattr(model, 'feature_names_in_'):
                features = model.feature_names_in_
                print(f"   Número de features entrenadas: {len(features)}")
                print(f"   Primeras 5 features: {list(features[:5])}")
            elif hasattr(model, 'n_features_'):
                print(f"   Número de features: {model.n_features_}")
            elif hasattr(model, 'n_features_in_'):
                print(f"   Número de features: {model.n_features_in_}")
        
        # Verificar scalers
        if 'scalers' in model_data:
            scalers = model_data['scalers']
            print(f"   Scalers disponibles: {list(scalers.keys()) if isinstance(scalers, dict) else 'No es dict'}")
            
            if isinstance(scalers, dict) and 'X' in scalers:
                scaler_X = scalers['X']
                if hasattr(scaler_X, 'n_features_in_'):
                    print(f"   Scaler X espera: {scaler_X.n_features_in_} features")
        
        # Verificar predicciones guardadas
        if 'predictions' in model_data:
            preds = model_data['predictions']
            if isinstance(preds, dict):
                print(f"   Predicciones guardadas: {list(preds.keys())}")
                if 'test' in preds:
                    test_preds = preds['test']
                    print(f"   Longitud predicciones test: {len(test_preds) if hasattr(test_preds, '__len__') else 'N/A'}")
        
        # Verificar features MIDAS
        if 'midas_features' in model_data:
            midas_feat = model_data['midas_features']
            if isinstance(midas_feat, dict):
                print(f"   Features MIDAS guardadas: {list(midas_feat.keys())}")
                if 'test' in midas_feat:
                    test_feat = midas_feat['test']
                    print(f"   Shape features MIDAS test: {test_feat.shape if hasattr(test_feat, 'shape') else 'N/A'}")
        
        # Verificar métricas
        if 'test_metrics' in model_data:
            metrics = model_data['test_metrics']
            if isinstance(metrics, dict):
                print(f"   Métricas disponibles: MAPE={metrics.get('mape', 'N/A'):.2f}%, R²={metrics.get('r2', 'N/A'):.3f}")
    
    print("-" * 60)


🔍 DIAGNÓSTICO DE MODELOS CARGADOS

📦 MIDAS_V2_hibrida:
   Tipo de datos: <class 'dict'>
   Claves disponibles: ['model', 'scalers', 'midas_features', 'best_params', 'train_metrics', 'test_metrics', 'predictions']
   Tipo de modelo: Ridge
   Número de features: 54
   Scalers disponibles: ['X', 'y']
   Scaler X espera: 54 features
   Predicciones guardadas: ['train', 'test']
   Longitud predicciones test: 281
   Features MIDAS guardadas: ['train', 'test']
   Shape features MIDAS test: (281, 54)
   Métricas disponibles: MAPE=0.26%, R²=0.974
------------------------------------------------------------

📦 MIDAS_V2_regime:
   Tipo de datos: <class 'dict'>
   Claves disponibles: ['model', 'scalers', 'midas_features', 'best_params', 'train_metrics', 'test_metrics', 'predictions']
   Tipo de modelo: Ridge
   Número de features: 58
   Scalers disponibles: ['X', 'y']
   Scaler X espera: 58 features
   Predicciones guardadas: ['train', 'test']
   Longitud predicciones test: 281
   Features MIDAS g

In [24]:
print("=" * 80)
print("🔮 USANDO PREDICCIONES PRE-GUARDADAS DE LOS MODELOS")
print("=" * 80)

# Diccionario para almacenar predicciones y métricas
predictions_alt = {}
metrics_results_alt = {}

print("\n🔍 Buscando predicciones guardadas en los modelos...")
print("-" * 60)

# IMPORTANTE: Las predicciones fueron hechas durante el entrenamiento
# sobre un período específico de test. Necesitamos identificar ese período.

for model_name, model_data in loaded_models.items():
    print(f"\n📈 Modelo: {model_name}")
    
    # Buscar predicciones pre-guardadas
    if isinstance(model_data, dict) and 'predictions' in model_data:
        preds = model_data['predictions']
        
        if isinstance(preds, dict) and 'test' in preds:
            y_pred = preds['test']
            
            # Verificar si es array o lista
            if hasattr(y_pred, '__len__'):
                y_pred = np.array(y_pred).flatten()
                n_pred = len(y_pred)
                print(f"   ✅ Predicciones encontradas: {n_pred} valores")
                
                # CORRECCIÓN CRÍTICA: Las predicciones corresponden a un período específico
                # del entrenamiento original, NO a los últimos días del dataset actual
                
                # Opción 1: Si queremos evaluar con los últimos 60 días
                # (esto requeriría re-entrenar o re-predecir)
                if False:  # Desactivado porque no tenemos esas predicciones
                    y_true = daily_data[TARGET_VAR].iloc[-60:].values
                    y_pred_subset = y_pred[-60:]  # Últimas 60 predicciones
                
                # Opción 2: Evaluar con el período original de test
                # Las predicciones corresponden a un conjunto de test del entrenamiento
                # Asumiendo split 80/20 del dataset en el momento del entrenamiento
                total_train_test = len(daily_data) - 60  # Excluir los últimos 60 días nuevos
                test_start_idx = int(total_train_test * 0.8)
                test_end_idx = test_start_idx + n_pred
                
                if test_end_idx <= len(daily_data):
                    y_true = daily_data[TARGET_VAR].iloc[test_start_idx:test_end_idx].values
                    dates = daily_data.index[test_start_idx:test_end_idx]
                    print(f"   📅 Período de test original: {dates[0].strftime('%Y-%m-%d')} a {dates[-1].strftime('%Y-%m-%d')}")
                else:
                    # Fallback: usar los últimos n_pred días disponibles
                    y_true = daily_data[TARGET_VAR].iloc[-n_pred:].values
                    dates = daily_data.index[-n_pred:]
                    print(f"   📅 Usando últimos {n_pred} días: {dates[0].strftime('%Y-%m-%d')} a {dates[-1].strftime('%Y-%m-%d')}")
                
                # Verificar longitudes
                if len(y_true) != len(y_pred):
                    min_len = min(len(y_true), len(y_pred))
                    y_true = y_true[:min_len]
                    y_pred = y_pred[:min_len]
                    dates = dates[:min_len]
                    print(f"   ⚠️ Ajustando longitudes a {min_len} valores")
                
                # Calcular métricas
                metrics = calculate_metrics(y_true, y_pred, model_name)
                
                # Almacenar resultados
                predictions_alt[model_name] = {
                    'y_true': y_true,
                    'y_pred': y_pred,
                    'dates': dates
                }
                metrics_results_alt[model_name] = metrics
                
                # Mostrar métricas
                print(f"   📊 MAPE: {metrics['mape']:.2f}%")
                print(f"   📊 RMSE: ${metrics['rmse']:.2f}")
                print(f"   📊 Hit Rate ±2%: {metrics['hit_rate_2pct']:.1f}%")
                if 'directional_accuracy' in metrics:
                    print(f"   📊 Directional Accuracy: {metrics['directional_accuracy']:.1f}%")
            else:
                print(f"   ⚠️ Predicciones en formato no reconocido")
        else:
            print(f"   ⚠️ No hay predicciones de test guardadas")
    else:
        # Si no hay predicciones guardadas, usar métricas pre-calculadas si existen
        if isinstance(model_data, dict) and 'test_metrics' in model_data:
            metrics = model_data['test_metrics']
            print(f"   📌 Usando métricas pre-calculadas:")
            print(f"   📊 MAPE: {metrics.get('mape', 'N/A'):.2f}%")
            print(f"   📊 RMSE: {metrics.get('rmse', 'N/A'):.4f}")
            print(f"   📊 Hit Rate ±2%: {metrics.get('hit_rate_2pct', 'N/A'):.1f}%")
            
            # Guardar métricas aunque no tengamos predicciones
            metrics_results_alt[model_name] = metrics
        else:
            print(f"   ❌ No hay predicciones ni métricas disponibles")

# Resumen de resultados alternativos
print("\n" + "=" * 80)
print("📊 RESUMEN DE EVALUACIÓN (PREDICCIONES PRE-GUARDADAS)")
print("=" * 80)

if len(metrics_results_alt) > 0:
    # Crear DataFrame con métricas
    metrics_df_alt = pd.DataFrame(metrics_results_alt).T
    metrics_df_alt = metrics_df_alt.round(4)
    
    print("\n🏆 Ranking por MAPE (menor es mejor):")
    ranking_alt = metrics_df_alt.sort_values('mape')
    for i, (model, row) in enumerate(ranking_alt.iterrows(), 1):
        print(f"   {i}. {model}:")
        print(f"      MAPE: {row['mape']:.2f}%")
        print(f"      RMSE: {row['rmse']:.4f}")
        if 'hit_rate_2pct' in row:
            print(f"      Hit Rate: {row['hit_rate_2pct']:.1f}%")
    
    # Mejor modelo
    best_model_name = ranking_alt.index[0]
    print(f"\n🥇 MEJOR MODELO: {best_model_name}")
    print(f"   MAPE: {ranking_alt.iloc[0]['mape']:.2f}%")
    
    # Actualizar variables globales si tuvimos éxito
    if len(predictions_alt) > 0:
        predictions = predictions_alt
        metrics_results = metrics_results_alt
        print(f"\n✅ Variables actualizadas con {len(predictions_alt)} modelos con predicciones")
    else:
        # Solo tenemos métricas, no predicciones
        metrics_results = metrics_results_alt
        print(f"\n⚠️ Solo métricas disponibles (sin predicciones para graficar)")
else:
    print("\n❌ No se encontraron predicciones ni métricas pre-guardadas")


🔮 USANDO PREDICCIONES PRE-GUARDADAS DE LOS MODELOS

🔍 Buscando predicciones guardadas en los modelos...
------------------------------------------------------------

📈 Modelo: MIDAS_V2_hibrida
   ✅ Predicciones encontradas: 281 valores
   📅 Período de test original: 2024-05-30 a 2025-06-26
   📊 MAPE: 3.17%
   📊 RMSE: $20.59
   📊 Hit Rate ±2%: 37.7%
   📊 Directional Accuracy: 54.6%

📈 Modelo: MIDAS_V2_regime
   ✅ Predicciones encontradas: 281 valores
   📅 Período de test original: 2024-05-30 a 2025-06-26
   📊 MAPE: 2.59%
   📊 RMSE: $17.33
   📊 Hit Rate ±2%: 47.0%
   📊 Directional Accuracy: 50.4%

📈 Modelo: XGBoost_V2_regime
   ✅ Predicciones encontradas: 286 valores
   📅 Período de test original: 2024-05-30 a 2025-07-03
   📊 MAPE: 4.30%
   📊 RMSE: $27.73
   📊 Hit Rate ±2%: 25.5%
   📊 Directional Accuracy: 49.5%

📈 Modelo: XGBoost_V2_hibrida
   ✅ Predicciones encontradas: 286 valores
   📅 Período de test original: 2024-05-30 a 2025-07-03
   📊 MAPE: 3.53%
   📊 RMSE: $23.23
   📊 Hit Rate ±

In [25]:
# 📈 VISUALIZACIÓN DE PREDICCIONES VS VALORES REALES

print("=" * 80)
print("📈 VISUALIZANDO PREDICCIONES")
print("=" * 80)

if len(predictions) > 0:
    # Crear figura con subplots
    n_models = len(predictions)
    fig = make_subplots(
        rows=n_models, 
        cols=1,
        subplot_titles=list(predictions.keys()),
        vertical_spacing=0.1
    )
    
    # Colores para cada modelo
    colors = ['blue', 'green', 'red', 'purple']
    
    for i, (model_name, pred_data) in enumerate(predictions.items(), 1):
        # Valores reales
        fig.add_trace(
            go.Scatter(
                x=pred_data['dates'],
                y=pred_data['y_true'],
                mode='lines',
                name='Real',
                line=dict(color='black', width=2),
                showlegend=(i == 1)
            ),
            row=i, col=1
        )
        
        # Predicciones
        fig.add_trace(
            go.Scatter(
                x=pred_data['dates'],
                y=pred_data['y_pred'],
                mode='lines',
                name='Predicción',
                line=dict(color=colors[i-1], width=2, dash='dot'),
                showlegend=(i == 1)
            ),
            row=i, col=1
        )
        
        # Agregar MAPE como anotación
        if model_name in metrics_results:
            mape = metrics_results[model_name]['mape']
            r2 = metrics_results[model_name]['r2']
            fig.add_annotation(
                text=f"MAPE: {mape:.2f}% | R²: {r2:.3f}",
                xref=f"x{i}",
                yref=f"y{i}",
                x=pred_data['dates'][-1],
                y=max(pred_data['y_true'].max(), pred_data['y_pred'].max()),
                showarrow=False,
                bgcolor="white",
                bordercolor="black",
                borderwidth=1
            )
    
    # Actualizar layout
    fig.update_layout(
        title="Predicciones vs Valores Reales - Últimos 60 días",
        height=300 * n_models,
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    
    # Actualizar ejes
    for i in range(1, n_models + 1):
        fig.update_xaxes(title_text="Fecha" if i == n_models else "", row=i, col=1)
        fig.update_yaxes(title_text="Precio ($)", row=i, col=1)
    
    fig.show()
    
    # Gráfico comparativo de errores
    print("\n📊 Comparación de Errores Absolutos")
    
    fig2 = go.Figure()
    
    for model_name, pred_data in predictions.items():
        errors = np.abs(pred_data['y_true'] - pred_data['y_pred'])
        fig2.add_trace(
            go.Scatter(
                x=pred_data['dates'],
                y=errors,
                mode='lines',
                name=model_name,
                line=dict(width=2)
            )
        )
    
    fig2.update_layout(
        title="Error Absoluto por Modelo",
        xaxis_title="Fecha",
        yaxis_title="Error Absoluto ($)",
        height=400,
        showlegend=True
    )
    
    fig2.show()
    
else:
    print("⚠️ No hay predicciones para visualizar")


📈 VISUALIZANDO PREDICCIONES



📊 Comparación de Errores Absolutos


In [27]:
# 📝 GENERAR REPORTE EJECUTIVO DE EVALUACIÓN

print("=" * 80)
print("📝 GENERANDO REPORTE EJECUTIVO")
print("=" * 80)

# Crear reporte en formato Markdown
report_lines = []
report_lines.append("# 📊 REPORTE DE EVALUACIÓN DE MODELOS - DeAcero Steel Price Predictor")
report_lines.append(f"\n**Fecha de Evaluación**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
report_lines.append(f"\n**Período de Test**: Últimos {test_size} días")
report_lines.append("\n---\n")

# Resumen Ejecutivo
report_lines.append("## 🎯 Resumen Ejecutivo\n")

if len(metrics_results) > 0:
    best_model = min(metrics_results.keys(), key=lambda x: metrics_results[x]['mape'])
    best_metrics = metrics_results[best_model]
    
    report_lines.append(f"### Mejor Modelo: **{best_model}**\n")
    report_lines.append(f"- **MAPE**: {best_metrics['mape']:.2f}%")
    report_lines.append(f"- **RMSE**: ${best_metrics['rmse']:.2f}")
    report_lines.append(f"- **Hit Rate (±2%)**: {best_metrics['hit_rate_2pct']:.1f}%")
    if 'directional_accuracy' in best_metrics:
        report_lines.append(f"- **Directional Accuracy**: {best_metrics['directional_accuracy']:.1f}%")

# Modelos Evaluados
report_lines.append("\n## 📈 Modelos Evaluados\n")
report_lines.append("| Modelo | MAPE (%) | RMSE ($) | Hit Rate (%) |")
report_lines.append("|--------|----------|----------|--------------|")

if len(metrics_results) > 0:
    for model_name in sorted(metrics_results.keys(), key=lambda x: metrics_results[x]['mape']):
        m = metrics_results[model_name]
        report_lines.append(f"| {model_name} | {m['mape']:.2f} | {m['rmse']:.2f} | {m['hit_rate_2pct']:.1f} |")

# Análisis Detallado
report_lines.append("\n## 📊 Análisis Detallado\n")

if len(metrics_results) > 0:
    # Análisis por tipo de modelo
    midas_models = [k for k in metrics_results.keys() if 'MIDAS' in k]
    xgboost_models = [k for k in metrics_results.keys() if 'XGBoost' in k]
    
    report_lines.append("### Por Arquitectura:\n")
    
    if midas_models:
        avg_mape_midas = np.mean([metrics_results[m]['mape'] for m in midas_models])
        report_lines.append(f"- **MIDAS Models**: MAPE promedio = {avg_mape_midas:.2f}%")
    
    if xgboost_models:
        avg_mape_xgb = np.mean([metrics_results[m]['mape'] for m in xgboost_models])
        report_lines.append(f"- **XGBoost Models**: MAPE promedio = {avg_mape_xgb:.2f}%")
    
    # Análisis por combinación
    report_lines.append("\n### Por Combinación de Variables:\n")
    
    hibrida_models = [k for k in metrics_results.keys() if 'hibrida' in k]
    regime_models = [k for k in metrics_results.keys() if 'regime' in k]
    
    if hibrida_models:
        avg_mape_hibrida = np.mean([metrics_results[m]['mape'] for m in hibrida_models])
        report_lines.append(f"- **Combinación Híbrida**: MAPE promedio = {avg_mape_hibrida:.2f}%")
    
    if regime_models:
        avg_mape_regime = np.mean([metrics_results[m]['mape'] for m in regime_models])
        report_lines.append(f"- **Combinación Régimen**: MAPE promedio = {avg_mape_regime:.2f}%")

# Recomendaciones
report_lines.append("\n## 💡 Recomendaciones\n")

if len(metrics_results) > 0:
    if best_metrics['mape'] < 2.0:
        report_lines.append("✅ **El modelo está listo para producción**")
        report_lines.append(f"   - MAPE < 2% indica excelente precisión para predicción de precios")
    elif best_metrics['mape'] < 5.0:
        report_lines.append("⚠️ **El modelo es aceptable pero puede mejorar**")
        report_lines.append(f"   - Considerar ajuste de hiperparámetros adicional")
    else:
        report_lines.append("❌ **Se recomienda continuar optimizando**")
        report_lines.append(f"   - MAPE > 5% puede ser insuficiente para decisiones críticas")
    
    if best_metrics['hit_rate_2pct'] > 70:
        report_lines.append("\n✅ **Alta precisión en predicciones cercanas**")
        report_lines.append(f"   - {best_metrics['hit_rate_2pct']:.0f}% de predicciones dentro del ±2%")

# Próximos Pasos
report_lines.append("\n## 🚀 Próximos Pasos\n")
report_lines.append("1. **Despliegue del Modelo Ganador**")
report_lines.append(f"   - Implementar {best_model if len(metrics_results) > 0 else 'modelo seleccionado'} en producción")
report_lines.append("2. **Monitoreo Continuo**")
report_lines.append("   - Establecer sistema de alertas para degradación de performance")
report_lines.append("3. **Reentrenamiento Periódico**")
report_lines.append("   - Actualizar modelo mensualmente con nuevos datos")
report_lines.append("4. **A/B Testing en Producción**")
report_lines.append("   - Comparar predicciones con modelo actual en paralelo")

# Guardar reporte
report_content = "\n".join(report_lines)

# Mostrar reporte
print(report_content)

# Guardar en archivo
report_dir = '../docs/04_MODEL_EVALUATION'
os.makedirs(report_dir, exist_ok=True)

report_filename = f"{report_dir}/evaluation_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
with open(report_filename, 'w') as f:
    f.write(report_content)

print(f"\n✅ Reporte guardado en: {report_filename}")

# Guardar métricas en JSON
if len(metrics_results) > 0:
    metrics_filename = f"{report_dir}/evaluation_metrics_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    
    # Preparar datos para JSON
    json_data = {
        'evaluation_date': datetime.now().isoformat(),
        'test_size': test_size,
        'best_model': best_model,
        'models': metrics_results
    }
    
    with open(metrics_filename, 'w') as f:
        json.dump(json_data, f, indent=2, default=str)
    
    print(f"✅ Métricas guardadas en: {metrics_filename}")


📝 GENERANDO REPORTE EJECUTIVO
# 📊 REPORTE DE EVALUACIÓN DE MODELOS - DeAcero Steel Price Predictor

**Fecha de Evaluación**: 2025-09-29 22:11:29

**Período de Test**: Últimos 60 días

---

## 🎯 Resumen Ejecutivo

### Mejor Modelo: **MIDAS_V2_regime**

- **MAPE**: 2.59%
- **RMSE**: $17.33
- **Hit Rate (±2%)**: 47.0%
- **Directional Accuracy**: 50.4%

## 📈 Modelos Evaluados

| Modelo | MAPE (%) | RMSE ($) | Hit Rate (%) |
|--------|----------|----------|--------------|
| MIDAS_V2_regime | 2.59 | 17.33 | 47.0 |
| MIDAS_V2_hibrida | 3.17 | 20.59 | 37.7 |
| XGBoost_V2_hibrida | 3.53 | 23.23 | 30.1 |
| XGBoost_V2_regime | 4.30 | 27.73 | 25.5 |

## 📊 Análisis Detallado

### Por Arquitectura:

- **MIDAS Models**: MAPE promedio = 2.88%
- **XGBoost Models**: MAPE promedio = 3.92%

### Por Combinación de Variables:

- **Combinación Híbrida**: MAPE promedio = 3.35%
- **Combinación Régimen**: MAPE promedio = 3.45%

## 💡 Recomendaciones

⚠️ **El modelo es aceptable pero puede mejorar**
   - Consider